In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

b = chat.predict("How many planets are there?")

print(b)

There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader  
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 600, 
    chunk_overlap=100,
    )


loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)
embedding = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(docs, embedding)


In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()


b = chat.predict("How many planets are there?")   #GPT 3.5 turbo 모델

b

'There are currently eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)   #temperature 가 높아질 수록 더 창의적이다

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage


messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in {language}.",
    ),
    AIMessage(content="Ciao, mi chiamo {name}!"),
    HumanMessage(
        content="What is the distance between {country_a} and {country_b}. Also, what is your name?",
    ),
]

chat.predict_messages(messages)

AIMessage(content='Mi dispiace, non posso calcolare la distanza tra due paesi senza informazioni più specifiche. Il mio nome è {name}, come posso aiutarti oggi?')

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)


template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [10]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)


chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [12]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [13]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

In [14]:

chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
#from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    #streaming=True,
    #callbacks=[
    #    StreamingStdOutCallbackHandler(),
    #],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [25]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain   # veg_chain 의 input을 chef_Chain 의 Output과 맞춰주는 과정정

final_chain.invoke({"cuisine": "indian"})

AIMessage(content='For a vegetarian twist on these traditional Indian recipes, here are some alternative ingredients you can use:\n\n1. Butter Chicken: Instead of chicken, you can use paneer (Indian cottage cheese) or tofu as a protein substitute. Marinate the paneer or tofu in a mixture of yogurt and spices before cooking it in the buttery tomato sauce.\n\n2. Chana Masala: To make this chickpea curry even more flavorful, you can add roasted vegetables like sweet potatoes or butternut squash. They will add a hint of sweetness and depth to the dish.\n\n3. Palak Paneer: For a vegan version of this dish, you can replace paneer with firm tofu or even boiled and cubed potatoes. The creamy spinach curry will still be delicious and nutritious.\n\n4. Biryani: Instead of meat, you can use a variety of vegetables like carrots, peas, bell peppers, and potatoes to make a flavorful vegetable biryani. Make sure to layer the rice and vegetables with fragrant spices for an aromatic dish.\n\n5. Aloo Go

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [29]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, chili powder, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or ov

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a suitable alternative such as paneer or tofu. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb paneer or extra-firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use plant-based yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground turmeric\n- 1 teaspoon garam masala\n- 1 teaspoon ground coriander\n- 1 teaspoon chili powder (adjust to taste)\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup heavy cream (you can use coconut cream for a vegan version)\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but use paneer or tofu instead of chicken. Marinate the paneer or tof

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),  
    ],
)


writing_Haiku_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international Haiku writer about programming languages. You write Haiku in a humorous style.",
        ),
        ("human", "I want to write a Haiku about {programming_language} ."),
    ]
)

writing_Haiku_chain = writing_Haiku_prompt | chat


explaining_Haiku_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are specialized in explaining Haiku. Provide the name of a programming language and respond with a haiku and its explanation.",
        ),
        ("human", "{haiku}"),
    ]
)

explaining_Haiku_chain = explaining_Haiku_prompt | chat


final_chain = {"haiku": writing_Haiku_chain} | explaining_Haiku_chain


final_chain.invoke({"programming_language": "c++"})



Pointers and classes,
Templates and inheritance,
C++ complexity.C++

Pointers and classes,
Templates and inheritance,
C++ complexity.

Explanation:
This haiku captures the essence of C++, highlighting some of its key features such as pointers, classes, templates, and inheritance. C++ is known for its complexity compared to other programming languages, but it also offers powerful tools and capabilities for building efficient and robust software systems.

AIMessageChunk(content='C++\n\nPointers and classes,\nTemplates and inheritance,\nC++ complexity.\n\nExplanation:\nThis haiku captures the essence of C++, highlighting some of its key features such as pointers, classes, templates, and inheritance. C++ is known for its complexity compared to other programming languages, but it also offers powerful tools and capabilities for building efficient and robust software systems.')

In [1]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

inputs = {"message": "hi im bob"}
response = chain.invoke(inputs)
response

ValueError: variable history should be a list of base messages, got {'history': []}

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Turkey"})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

In [ ]:
# Cost 를 줄이기 위해 example selector 사용법을 익혀야함. 4.3강

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum Soup
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum Soup\n        Currency: Thai Baht\n        ')

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [13]:
# 프롬프트를 loading해서 쓰는 것도 알아야한다 4.4 장
# 여러 프롬프트를 하나로 합치는 기술 (2번째 과제에 써야할듯)
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "character": "Pirate",
  "example_question": "What is your location?",
  "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
  "question": "What is your fav food?"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:PipelinePromptTemplate] Entering Prompt run with input:
{
  "character": "Pirate",
  "example_question": "What is your location?",
  "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
  "question": "What is your fav food?"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:PipelinePromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n 

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!")

In [ ]:
# 캐시는 언어모델의 응답을 저장할 수 있다
# 같은 질문을 받을 때 돈을 줄여주기 위해
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache  # DB에 저장할 수 있다, 케시에 저장할 수도 있고

set_llm_cache(SQLiteCache("cache.db")) 
# 이름은 마음대로 해도 된다
#set_debug(True)   # 한번 켜면 꺼지지가 않네 디버깅 할때 유용


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a t

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the pasta sheets into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain

In [ ]:
#얼마나 비용이 드는지 보는것것
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [2.03s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of water\n- 1 tablespoon of nuruk (fermentation starter)\n- 1 tablespoon of sugar\n\nInstructions:\n1. Rinse the rice thoroughly and soak it in water for at least 1 hour.\n2. Drain the rice and steam it until it is fully cooked.\n3. Let the rice cool down to room temperature.\n4. In a large bowl, mix the nuruk and sugar with the cooked rice.\n5. Cover the bowl with a clean cloth and let it ferment for 3-4 days in a warm place.\n6. After fermentation, strain the mixture through a cheesecloth to remove any solid particles.\n7. Transfer the liquid to a clean container and let it sit for another 1-2 days to allow the flavors to develop.\n8. Your homemade soju is now ready to be enjoyed! Serve chilled and enjoy res

In [7]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or rolling pin.\n7. Cut the pasta sheets into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain